# DATA WRANGLING

C/W = Comply with

> C/W - Using the repo setup directions, setup a new local and remote repository named clustering-exercises s. The local version of your repo should live inside of ~/codeup-data-science. This repo should be named clustering-exercises

> C/W - Save your clustering work in your clustering-exercises repo. Then add, commit, and push your changes.

> C/W - For example, if the exercise directs you to create a file named myfile.py, you should have clustering/myfile.py in your repository.

> C/W - If a file extension is specified, you should create that specific file. If there is not file extension specified, you may either create a python script or a jupyter notebook for the exercise.

Throughout the exercises, you may wish to do your work in a notebook, then transfer any functions you've created to an external python script.

Keep in mind this is not always a linear process! You will probably be cycling between a notebook and an external python script frequently.

Remember to run your code often to check for correct output and/or errors.

### Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

## IMPORT

In [1]:
# Standard imports
import numpy as np
import pandas as pd

# My imports
import wrangle as w

from pydataset import data
from scipy import stats


import matplotlib.pyplot as plt
import seaborn as sns

# # Stats
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import TweedieRegressor
# from sklearn.linear_model import LassoLars
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Acquire and Summarize

Acquire data from the cloud database. - C/W

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database. - C/W

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - C/W

Only include properties that have a latitude and longitude value. C/W

Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.) - C/W


In [2]:
df = w.get_zillow_data()

csv file found and loaded


In [3]:
df.shape

(77613, 69)

In [4]:
# sort values by transactiondate
df = df.sort_values(by='transactiondate', ascending=False)

# Drop duplicate rows in column: 'parcelid', keeping last transaction date
df = df.drop_duplicates(subset=['parcelid'], keep='last')

# drop all properties that have a latitude and longitude values
# this will drop all the null from the latitude and longitude columns
# only dropped 232 rows
df = df.dropna(subset=['latitude','longitude'])

In [5]:
# 232 rows were dropped from the latitude and longitude columns
df.shape

(77381, 69)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 77612 to 0
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   heatingorsystemtypeid         49440 non-null  float64
 3   buildingclasstypeid           15 non-null     float64
 4   architecturalstyletypeid      206 non-null    float64
 5   airconditioningtypeid         24953 non-null  float64
 6   parcelid                      77381 non-null  int64  
 7   propertylandusetypeid         77381 non-null  float64
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

In [7]:
cols_to_remove = ['typeconstructiontypeid','storytypeid','heatingorsystemtypeid','buildingclasstypeid'
                  ,'architecturalstyletypeid','airconditioningtypeid','propertylandusetypeid','id','pooltypeid10'
                  ,'pooltypeid2','pooltypeid7','rawcensustractandblock','regionidcity','regionidcounty'
                 ,'regionidneighborhood','regionidzip','censustractandblock','id.1','landtaxvaluedollarcnt'
                 ,'taxamount']

In [8]:
df = w.remove_columns(df, cols_to_remove)
df

,parcelid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxdelinquencyyear,propertylandusedesc,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
77612,12826780,NaN,2.0,3.0,6.0,2.0,NaN,NaN,1762.0,1762.0,...,NaN,Single Family Residential,0.007204,2017-09-25,NaN,NaN,NaN,Central,NaN,NaN
77611,12773139,NaN,1.0,3.0,4.0,1.0,NaN,NaN,1032.0,1032.0,...,NaN,Single Family Residential,0.037129,2017-09-21,Central,NaN,NaN,Central,NaN,NaN
77610,17239384,NaN,2.0,4.0,NaN,2.0,NaN,1612.0,1612.0,1612.0,...,NaN,Single Family Residential,0.013209,2017-09-21,NaN,NaN,NaN,NaN,NaN,NaN
77609,11000655,NaN,2.0,2.0,6.0,2.0,NaN,NaN,1286.0,1286.0,...,NaN,Single Family Residential,0.020615,2017-09-20,NaN,NaN,NaN,Central,NaN,NaN
77608,10833991,NaN,3.0,3.0,8.0,3.0,NaN,NaN,1741.0,1741.0,...,NaN,Condominium,-0.002245,2017-09-20,Central,NaN,NaN,Central,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,17143294,NaN,2.0,3.0,NaN,2.0,NaN,842.0,1492.0,1492.0,...,NaN,Condominium,-0.020526,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN
4,10887214,NaN,3.0,3.0,8.0,3.0,NaN,NaN,1312.0,1312.0,...,NaN,Condominium,0.006940,2017-01-01,Central,NaN,NaN,Central,NaN,NaN
3,12177905,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,...,NaN,Single Family Residential,-0.103410,2017-01-01,NaN,NaN,NaN,Central,NaN,NaN
2,14186244,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,...,NaN,Single Family Residential,0.005383,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN


### Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values. - C/W

The output should look like the table below:

In [13]:
cols_missing = w.nulls_by_col(df)
cols_missing

,num_rows_missing,percent_rows_missing
parcelid,0,0.000000
basementsqft,77331,99.935385
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
buildingqualitytypeid,27709,35.808532
calculatedbathnbr,609,0.787015
decktypeid,76767,99.206524
finishedfloor1squarefeet,71358,92.216436
calculatedfinishedsquarefeet,196,0.253292
finishedsquarefeet12,3632,4.693659


In [12]:
rows_missing = w.nulls_by_row(df, index_id = 'parcelid')
rows_missing

,parcelid,num_cols_missing,percent_cols_missing
22116,14341728,36,73.469388
38462,14356698,36,73.469388
41819,14696057,35,71.428571
64364,167686999,35,71.428571
32137,14714041,35,71.428571
...,...,...,...
14836,17292665,17,34.693878
1553,17208897,17,34.693878
58476,17097730,17,34.693878
62137,17097816,16,32.653061


In [ ]:
object_cols = w.get_object_cols(df)

In [ ]:
num_cols = w.get_numeric_cols(df)

In [ ]:
w.summarize(df)

In [ ]:
num_cols = len(w.get_numeric_cols(df))
num_rows, num_cols_subplot = divmod(num_cols, 3)
if num_cols_subplot > 0:
    num_rows += 1

fig, axes = plt.subplots(num_rows, 3, figsize=(15, num_rows * 5))

for i, col in enumerate(w.get_numeric_cols(df)):
    row_idx, col_idx = divmod(i, 3)
    sns.histplot(df[col], ax=axes[row_idx, col_idx])
    axes[row_idx, col_idx].set_title(f'Histogram of {col}')

plt.tight_layout()
plt.show()

In [ ]:
# # fig, axes = plt.subplots(5, len(w.get_numeric_cols(df)), figsize=(15, 5))

# for i, col in enumerate(w.get_numeric_cols(df)):
#     sns.histplot(df[col], ax = axes[i])
#     axes[i].set_title(f'Histogram of {col}')
# plt.show()

for col in w.get_numeric_cols(df):
    sns.histplot(df[col])
    plt.show()

In [ ]:
df = w.handle_missing_values(df, prop_required_columns=0.5, prop_required_rows=0.75)